In [1]:
import pandas as pd
import numpy as np

In [2]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [21]:
donor = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/education project/data/clean_essay.csv")
essay_lda = donor[['resource_type','primary_focus_area', 'poverty_level','text']]
essay_lda['doc'] = essay_lda['text'].apply(lambda x: x.split(" "))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
essay_lda.loc[(essay_lda['poverty_level'] == 'highest poverty'), "poor"] = 1
essay_lda.poor.fillna(0,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.or

In [26]:
from sklearn.model_selection import train_test_split
essay_train, essay_test = train_test_split(essay_lda, test_size=0.3, random_state=1234)

In [27]:
from gensim import corpora, models, similarities
dictionary = corpora.Dictionary(essay_train.doc)
corpus = [dictionary.doc2bow(text) for text in essay_train.doc]
# print(dictionary)
# print([[(dictionary[id],freq) for id, freq in cp] for cp in corpus[:1]])
tfidf = models.TfidfModel(corpus)
transformed_tfidf = tfidf[corpus]

import gensim
import logging # This allows for seeing if the model converges. A log file is created.
logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

lda_train = gensim.models.ldamulticore.LdaMulticore(
                       corpus=transformed_tfidf,
                       num_topics=10,
                       id2word=dictionary,
                       per_word_topics=True)
lda_train.save('lda_train.model')

In [28]:
lda_train.print_topics(num_words=15)

[(0,
  '0.002*"book" + 0.002*"technology" + 0.002*"math" + 0.002*"ipod" + 0.002*"computer" + 0.002*"read" + 0.002*"calculator" + 0.001*"science" + 0.001*"reading" + 0.001*"skill" + 0.001*"listen" + 0.001*"library" + 0.001*"center" + 0.001*"level" + 0.001*"child"'),
 (1,
  '0.002*"book" + 0.002*"read" + 0.001*"technology" + 0.001*"science" + 0.001*"ipad" + 0.001*"computer" + 0.001*"reading" + 0.001*"math" + 0.001*"child" + 0.001*"game" + 0.001*"center" + 0.001*"kid" + 0.001*"use" + 0.001*"skill" + 0.001*"listen"'),
 (2,
  '0.003*"book" + 0.002*"read" + 0.002*"reading" + 0.001*"listen" + 0.001*"music" + 0.001*"story" + 0.001*"center" + 0.001*"technology" + 0.001*"write" + 0.001*"child" + 0.001*"cd" + 0.001*"math" + 0.001*"word" + 0.001*"art" + 0.001*"kid"'),
 (3,
  '0.004*"book" + 0.003*"read" + 0.002*"reading" + 0.002*"computer" + 0.002*"child" + 0.002*"science" + 0.002*"technology" + 0.002*"camera" + 0.002*"math" + 0.002*"classroom" + 0.002*"material" + 0.002*"library" + 0.002*"art" + 

In [35]:
train_vecs = []
for i in range(essay_train.shape[0]):
    top_topics = (
        lda_train.get_document_topics(transformed_tfidf[i],
                                      minimum_probability=0.0)
    )
    topic_vec = [top_topics[i][1] for i in range(10)]
    topic_vec.extend([len(essay_train.iloc[i].text)])
    train_vecs.append(topic_vec)

In [44]:
X = np.array(train_vecs)
y = np.array(essay_train.poor)
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score 
kf = KFold(5, shuffle=True, random_state=1234)
cv_lr_f1, cv_lrsgd_f1, cv_svcsgd_f1,  = [], [], []

for train_ind, val_ind in kf.split(X, y):
    # Assign CV IDX
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind]
    
    # Scale Data
    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_val_scale = scaler.transform(X_val)

    # Logisitic Regression
    lr = LogisticRegression(
        class_weight= 'balanced',
        solver='newton-cg',
        fit_intercept=True
    ).fit(X_train_scale, y_train)

    y_pred = lr.predict(X_val_scale)
    cv_lr_f1.append(f1_score(y_val, y_pred, average='binary'))
    
    # Logistic Regression SGD
    sgd = SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        loss='log',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)
    
    y_pred = sgd.predict(X_val_scale)
    cv_lrsgd_f1.append(f1_score(y_val, y_pred, average='binary'))
    
    # SGD Modified Huber
    sgd_huber = SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        alpha=20,
        loss='modified_huber',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)
    
    y_pred = sgd_huber.predict(X_val_scale)
    cv_svcsgd_f1.append(f1_score(y_val, y_pred, average='binary'))

print(f'Logistic Regression Val f1: {np.mean(cv_lr_f1):.3f} +- {np.std(cv_lr_f1):.3f}')
print(f'Logisitic Regression SGD Val f1: {np.mean(cv_lrsgd_f1):.3f} +- {np.std(cv_lrsgd_f1):.3f}')
print(f'SVM Huber Val f1: {np.mean(cv_svcsgd_f1):.3f} +- {np.std(cv_svcsgd_f1):.3f}')

Logistic Regression Val f1: 0.598 +- 0.048
Logisitic Regression SGD Val f1: 0.633 +- 0.081
SVM Huber Val f1: 0.323 +- 0.395


## Test

In [46]:
test_vecs = []
for i in range(essay_test.shape[0]):
    top_topics = (
        lda_train.get_document_topics(transformed_tfidf[i],
                                      minimum_probability=0.0)
    )
    topic_vec = [top_topics[i][1] for i in range(10)]
    topic_vec.extend([len(essay_test.iloc[i].text)])
    test_vecs.append(topic_vec)

In [47]:
X = np.array(test_vecs)
y = np.array(essay_test.poor)
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score 
kf = KFold(5, shuffle=True, random_state=1234)
cv_lr_f1, cv_lrsgd_f1, cv_svcsgd_f1,  = [], [], []

for train_ind, val_ind in kf.split(X, y):
    # Assign CV IDX
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind]
    
    # Scale Data
    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_val_scale = scaler.transform(X_val)

    # Logisitic Regression
    lr = LogisticRegression(
        class_weight= 'balanced',
        solver='newton-cg',
        fit_intercept=True
    ).fit(X_train_scale, y_train)

    y_pred = lr.predict(X_val_scale)
    cv_lr_f1.append(f1_score(y_val, y_pred, average='binary'))
    
    # Logistic Regression SGD
    sgd = SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        loss='log',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)
    
    y_pred = sgd.predict(X_val_scale)
    cv_lrsgd_f1.append(f1_score(y_val, y_pred, average='binary'))
    
    # SGD Modified Huber
    sgd_huber = SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        alpha=20,
        loss='modified_huber',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)
    
    y_pred = sgd_huber.predict(X_val_scale)
    cv_svcsgd_f1.append(f1_score(y_val, y_pred, average='binary'))

print(f'Logistic Regression Val f1: {np.mean(cv_lr_f1):.3f} +- {np.std(cv_lr_f1):.3f}')
print(f'Logisitic Regression SGD Val f1: {np.mean(cv_lrsgd_f1):.3f} +- {np.std(cv_lrsgd_f1):.3f}')
print(f'SVM Huber Val f1: {np.mean(cv_svcsgd_f1):.3f} +- {np.std(cv_svcsgd_f1):.3f}')

Logistic Regression Val f1: 0.573 +- 0.049
Logisitic Regression SGD Val f1: 0.640 +- 0.088
SVM Huber Val f1: 0.489 +- 0.399
